<a href="https://colab.research.google.com/github/bryonbaker/SIT720/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SIT720 - Assignment 1

Bryon Baker
Student number: 85031775

# Import required modules and libraries

In [1]:

import pandas as pd
import numpy as np

# Function Declarations

In [2]:
# This function calculates a value for the missing gender feature. 
# If the person is pregnant it must be female. If not pregnant then it defaults to male.
# REVISIT: Update the non-pregnant case after you get an answer from the lecturer.
def calc_gender(pregnant: str):
    if pregnant == "t":
        gender = "F"
    else:
        gender = "M"
    return gender

# Load Dataset
Load the dataset from a public git repo.
Display the head info to confirm data load.

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

url = 'https://raw.githubusercontent.com/bryonbaker/datasets/main/SIT720/Ass1/hypothyroid.csv'
fullht_df = pd.read_csv(url)

In [4]:
fullht_df.head(n=100)

,age,sex,on thyroxine,pregnant,I131 treatment,query hyperthyroid,TSH,T3,TT4,FTI,binaryClass
0,41,F,f,f,f,f,1.3,2.5,125,109,P
1,23,F,f,f,f,f,4.1,2,102,?,P
2,46,M,f,f,f,f,0.98,?,109,120,P
3,70,F,t,f,f,f,0.16,1.9,175,?,P
4,70,F,f,f,f,f,0.72,1.2,61,70,P
...,...,...,...,...,...,...,...,...,...,...,...
95,26,F,f,f,f,f,2,?,137,84,P
96,59,?,f,f,f,f,1.5,?,92,99,P
97,39,F,f,f,f,f,4.3,2.4,135,89,P
98,69,F,f,f,f,f,1.4,?,123,136,P


# Data Preparation

## Task 1
1.   Select the first 500 rows of the dataset
2.   Display the dataset dimension



In [13]:
# Get the first 500 rows from the dataset and use that for the rest of the assignment.
ht_df = fullht_df.head(n=500)

# Display the dataset's dimension
print(f"Working dataset dimension is: {ht_df.shape}\n")

Working dataset dimension is: (500, 11)



## Task 2
* Display the number of rows and their indices that have missing data in one or more cells. 
* Print the median values of all features before and after replacing missing data.


In [14]:
# Cells with missing data have a '?' in them. 
# First replace ? with np.NaN so we can utilise some other nice Pandas dataframe methods. We can use a global replace because, upon dataset ins[ection, the unknown ('?') only exists in the numeric columns.
# Convert the value columns from text to numeric.
# Calculate the median value for the numeric-data coluimns
# Replace the NaN values with a reasonable value. For this exercise we have chosen the mean for the column
# Recalculate the median value for the numeric-data coluimns

# Prepare the data so it is calculable
ht_df = ht_df.replace('?', np.NaN)                                                        # Replace with NaN so many of the Pandas functions will work.
ht_df[["TSH","T3","TT4","FTI"]] = ht_df[["TSH","T3","TT4","FTI"]].apply(pd.to_numeric)    # CSV loads as text. Convert the cells to numeric
print("\nCore Numerical Data Columns:\n")
print(ht_df[["TSH","T3","TT4","FTI"]])

# Calculate the Mean and Median prior to replacing missing values
mean = ht_df[["TSH","T3","TT4","FTI"]].mean(skipna=True)
median = ht_df[["TSH","T3","TT4","FTI"]].median(skipna=True)
print("=======================\n")
print("\nPre Data Modification:")
print("\n======================")
print(f"Mean of each column is:\n{mean}\n")
print(f"The Median of each column is:\n{median}")

# Replace the NaN's of the numeric columns with the mean
ht_df["TSH"] = ht_df["TSH"].fillna(mean["TSH"])
ht_df["T3"] = ht_df["TSH"].fillna(mean["T3"])
ht_df["TT4"] = ht_df["TSH"].fillna(mean["TT4"])
ht_df["FTI"] = ht_df["TSH"].fillna(mean["FTI"])

# Replace the M/F missing values with the most frequently occuring gender provided "pregnant" is false. Otherwise set the value to F.
# replace_missing_values(df=ht_df, col="sex", v1="M", v2="F")
tmp_col = "sex-predict"
ht_df[tmp_col] = ht_df["sex"]
for (index, row_series) in ht_df.iterrows():
    if pd.isna(row_series["sex"]):
#        print(f"DEBUG: Found NaN at: {index}")
#        print(f"DEBUG: {ht_df.iloc[index]}")
        ht_df.at[index, tmp_col] = calc_gender(ht_df.at[index, 'pregnant'])
#        print(f"DEBUG: {type(ht_df.iloc[index])}")

# Copy over any NaN values in the sex column using the value from the temporary column
ht_df["sex"] = ht_df["sex"].fillna(ht_df[tmp_col])
ht_df = ht_df.drop([tmp_col], axis=1)       # Drop the temporary column

mean_post = ht_df[["TSH","T3","TT4","FTI"]].mean(skipna=True)
median_post = ht_df[["TSH","T3","TT4","FTI"]].median(skipna=True)
print("=======================")
print("Post Data Modification:")
print("=======================")
#print(f"Mean of each column is:\n{mean_post}")
print(f"The Median is:\n{median_post}")


Core Numerical Data Columns:

       TSH   T3    TT4    FTI
0     1.30  2.5  125.0  109.0
1     4.10  2.0  102.0    NaN
2     0.98  NaN  109.0  120.0
3     0.16  1.9  175.0    NaN
4     0.72  1.2   61.0   70.0
..     ...  ...    ...    ...
495    NaN  NaN   93.0  102.0
496   0.02  1.4  144.0    NaN
497   0.15  1.4  133.0  134.0
498  11.00  1.5  144.0  116.0
499   0.10  NaN   62.0   66.0

[500 rows x 4 columns]


Pre Data Modification:

Mean of each column is:
TSH      4.523496
T3       1.950875
TT4    107.305263
FTI    110.084382
dtype: float64

The Median of each column is:
TSH      1.4
T3       1.9
TT4    103.0
FTI    108.0
dtype: float64
   age  sex on thyroxine pregnant I131 treatment query hyperthyroid   TSH  \
0   41    F            f        f              f                  f  1.30   
1   23    F            f        f              f                  f  4.10   
2   46    M            f        f              f                  f  0.98   
3   70    F            t        f         